In [7]:
import pandas as pd
import numpy as np
import random

# ============= تنظیمات GA و Leader Correction ============
POP_SIZE = 30
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.12
N_GENERATIONS = 50
ELITE_SIZE = 2
WINDOW_SIZE = 200

MODELS = ['PPPL', 'dSMA']
PARAM_BOUNDS = {
    'PPPL': {'n_SMA': (14, 20), 'd11': (0.05, 0.2), 'SL': (50, 150), 'TP': (50, 200)},
    'dSMA': {'n_SMA': (14, 20), 'd11': (0.05, 0.2), 'SL': (50, 150), 'TP': (50, 200)}
}

# ================= توابع محاسبات =================
def ValueATR(df, period):
    df['ValueATR'] = (df['High'] - df['Low']).rolling(window=period).mean()
    return df

def avgHL(df, period):
    df['avgHL'] = (df['High'] - df['Low']).rolling(window=period).mean()
    return df

def pp_pl(df, period):
    df['d'] = df['Close'] - df['Open']
    df['pp'] = df['d'].apply(lambda x: x if x > 0 else 0).rolling(period).mean()
    df['pl'] = df['d'].apply(lambda x: abs(x) if x < 0 else 0).rolling(period).mean()
    df['dP'] = abs(df['pp'] - df['pl']) / df[['pp', 'pl']].max(axis=1)
    return df

def DSMA(df, period):
    df['aa'] = df['Open']
    df['bb'] = df['Close'].rolling(period).mean()
    df['dSMA'] = df['aa'] - df['bb']
    df['dD'] = abs(df['dSMA']) / df[['aa', 'bb']].max(axis=1)
    return df

def Setup_Sell_Buy(flag_buy, flag_sell, Open, avgHL_val, wsp_Sl, wsp_Tp):
    dSl, dTp = avgHL_val * wsp_Sl, avgHL_val * wsp_Tp
    Sl, Tp = None, None
    if flag_buy == 1:
        Sl = Open - dSl
        Tp = Open + dTp
    elif flag_sell == 1:
        Sl = Open + dSl
        Tp = Open - dTp
    return Sl, Tp

# ================= بک‌تست با استفاده از iloc =================
def backtest_model(model_name, params, data):
    n_sma, d11, wsp_Sl, wsp_Tp = params
    df = data.copy()
    pp_pl(df, int(n_sma))
    DSMA(df, int(n_sma))
    ValueATR(df, int(n_sma))
    avgHL(df, int(n_sma))

    df['dd1'] = abs(df['dP'] - df['dD'])
    df['BSPower'] = df['dP']

    flag_buy, flag_sell = 0, 0
    net_profit, trades = 0, 0

    for i in range(1, len(df)):
        if 7 <= df['Hour'].iloc[i] < 19 and (flag_buy + flag_sell) == 0:
            if df['ValueATR'].iloc[i-1] > 0 and df['BSPower'].iloc[i-1] > 0:
                if model_name == 'PPPL' and df['dSMA'].iloc[i-1] > 0 and df['pp'].iloc[i-1] > df['pl'].iloc[i-1]:
                    flag_buy = 1
                    Sl, Tp = Setup_Sell_Buy(flag_buy, flag_sell, df['Open'].iloc[i], df['avgHL'].iloc[i-1], wsp_Sl, wsp_Tp)
                    trades += 1
                    if df['Low'].iloc[i] <= Sl:
                        net_profit += Sl - df['Open'].iloc[i]
                        flag_buy = 0
                    elif df['High'].iloc[i] >= Tp:
                        net_profit += Tp - df['Open'].iloc[i]
                        flag_buy = 0

                elif model_name == 'dSMA' and df['dSMA'].iloc[i-1] < 0 and df['pp'].iloc[i-1] < df['pl'].iloc[i-1]:
                    flag_sell = 1
                    Sl, Tp = Setup_Sell_Buy(flag_buy, flag_sell, df['Open'].iloc[i], df['avgHL'].iloc[i-1], wsp_Sl, wsp_Tp)
                    trades += 1
                    if df['High'].iloc[i] >= Sl:
                        net_profit += df['Open'].iloc[i] - Sl
                        flag_sell = 0
                    elif df['Low'].iloc[i] <= Tp:
                        net_profit += df['Open'].iloc[i] - Tp
                        flag_sell = 0

    net_profit *= 100000
    avg_profit = net_profit / trades if trades > 0 else 0
    avg_loss = abs(avg_profit * d11)
    profit_risk_ratio = avg_profit / avg_loss if avg_loss else 0
    equity_curve = df['Close'].cummax() - df['Close']
    drawdown = equity_curve.max() / df['Close'].max()
    return net_profit, profit_risk_ratio, drawdown

# ================= GA استاندارد =================
def initialize_population(bounds):
    return [[random.uniform(bounds[key][0], bounds[key][1]) for key in bounds] for _ in range(POP_SIZE)]

def evaluate_population(pop, objective_func):
    return [objective_func(ind) for ind in pop]

def tournament_selection(pop, scores, k=3):
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(zip(pop, scores)), k)
        selected.append(max(aspirants, key=lambda x: x[1])[0])
    return selected

def crossover(p1, p2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, len(p1) - 1)
        return p1[:point] + p2[point:], p2[:point] + p1[point:]
    return p1, p2

def mutate(ind, bounds):
    for i, key in enumerate(bounds):
        if random.random() < MUTATION_RATE:
            ind[i] = random.uniform(bounds[key][0], bounds[key][1])
    return ind

def elitism_selection(pop, scores):
    elite_idx = np.argsort(scores)[-ELITE_SIZE:]
    return [pop[i] for i in elite_idx]

# ================= Leader Correction با ایندکس ریست‌شده =================
def genetic_algorithm_leader_correction(data):
    results = []
    for start in range(0, len(data) - WINDOW_SIZE, WINDOW_SIZE):
        window_data = data.iloc[start:start+WINDOW_SIZE].reset_index(drop=True)

        best_models, base_metrics, best_scores = {}, {}, {}
        for model in MODELS:
            bounds = PARAM_BOUNDS[model]
            pop = initialize_population(bounds)
            for _ in range(N_GENERATIONS):
                scores = evaluate_population(pop, lambda p: backtest_model(model, p, window_data)[0])
                elite = elitism_selection(pop, scores)
                selected = tournament_selection(pop, scores)
                children = []
                for i in range(0, POP_SIZE, 2):
                    c1, c2 = crossover(selected[i], selected[(i+1) % POP_SIZE])
                    children.append(mutate(c1, bounds))
                    children.append(mutate(c2, bounds))
                pop = elite + children[:POP_SIZE-ELITE_SIZE]
            final_scores = evaluate_population(pop, lambda p: backtest_model(model, p, window_data)[0])
            best_idx = np.argmax(final_scores)
            best_models[model] = pop[best_idx]
            best_scores[model] = final_scores[best_idx]
            base_metrics[model] = backtest_model(model, pop[best_idx], window_data)

        leader_model = max(best_scores, key=best_scores.get)
        leader_params = best_models[leader_model]
        net_profit, profit_risk, drawdown = base_metrics[leader_model]

        before_np = np.mean([m[0] for m in base_metrics.values()])
        before_pr = np.mean([m[1] for m in base_metrics.values()])
        before_dd = np.mean([m[2] for m in base_metrics.values()])

        results.append({
            'Window_Start': start,
            'Leader_Model': leader_model,
            'n_SMA': leader_params[0], 'd11': leader_params[1],
            'SL': leader_params[2], 'TP': leader_params[3],
            'Net_Profit': net_profit,
            'Profit/Risk': profit_risk,
            'Drawdown': drawdown,
            '%Improvement_NP': (net_profit - before_np) / abs(before_np) * 100 if before_np else 0,
            '%Improvement_PR': (profit_risk - before_pr) / abs(before_pr) * 100 if before_pr else 0,
            '%Improvement_DD': (before_dd - drawdown) / abs(before_dd) * 100 if before_dd else 0
        })
    return pd.DataFrame(results)

# ================= اجرای یک‌بار =================
if __name__ == "__main__":
    df_train = pd.read_csv("EURUSD_H4.csv")
    df_train['Date'] = pd.to_datetime(df_train['Date'])
    df_train['Hour'] = pd.to_datetime(df_train['Time'], format='%H:%M:%S').dt.hour

    df_test = pd.read_csv("USDCHF_H4.csv")
    df_test['Date'] = pd.to_datetime(df_test['Date'])
    df_test['Hour'] = pd.to_datetime(df_test['Time'], format='%H:%M:%S').dt.hour

    train_results = genetic_algorithm_leader_correction(df_train)
    train_results.to_excel("LC_results_train.xlsx", index=False)

    test_results = genetic_algorithm_leader_correction(df_test)
    test_results.to_excel("LC_results_test.xlsx", index=False)

    print("✅ خروجی‌ها ساخته شدند و ذخیره شدند.")
    print("فایل‌ها آماده: LC_results_train.xlsx و LC_results_test.xlsx")


✅ خروجی‌ها ساخته شدند و ذخیره شدند.
فایل‌ها آماده: LC_results_train.xlsx و LC_results_test.xlsx
